In [0]:
%run ../DataProcesing/Transversal/config

In [0]:
%run ../DataProcesing/Transversal/utils

In [0]:
training_dataset = spark.table(Gold_Train_Model_Dataset)

In [0]:
from datetime import datetime, timedelta

tomorrow = datetime.now(horario_colombia) + timedelta(days=1)

day_number_tomorrow = (tomorrow.weekday() + 2)

if day_number_tomorrow > 7:
    day_number_tomorrow -= 7

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number, lit


window = Window.partitionBy("neighborhood").orderBy(col("event_date").desc())

df_latest = training_dataset.withColumn("row_num", row_number().over(window))\
                            .filter(col("row_num") == 1)\
                            .drop("row_num")

df_pred = df_latest.withColumn("day_number", lit(day_number_tomorrow))\
                   .drop(*["event_date", "quantity_products"])

In [0]:
import mlflow.pyfunc
from pyspark.ml import PipelineModel

# Load the model as Pyfunc
model_uri = "/Volumes/unalwater_v2/default/files/Model"
# Model = mlflow.pyfun.load_model (Model_uri)
model = mlflow.spark.load_model(model_uri)

df_result  = model.transform(df_pred)

In [0]:
# Training_dataset_pd = Training_dataset.Topandas ()
predictions = model.transform(training_dataset)

bias_by_neighborhood = calculate_bias_by_neighborhood(dataset=predictions)

In [0]:
df_result_with_bias = df_result.join(
    bias_by_neighborhood.select("neighborhood", "avg_bias", "bias_percentage"),
    on="neighborhood",
    how="left"
)

df_final = df_result_with_bias.select(
    "neighborhood",
    "prediction",
    "avg_bias",
    "bias_percentage"
)

In [0]:
df_final.write.format('delta').mode("overwrite").save(Gold_Demand_Prediction)

In [0]:
df_final.write.mode("overwrite").saveAsTable("gold_demand_prediction")
